*Goals of this homework:* 

# Part 1. Collaborative Filtering

In this part, you will implement collaborative filtering on the [MovieLens Latest Datasets](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html). After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

As background, read [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf) and refer to the course slides `week06rec.pdf` for collaborative filtering.

## 1.1 Load the Data

Please download the dataset.There are about 65,000 ratings in total. We split the rating data into two set. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [1]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...
import numpy as np
import pandas as pd
import json
import math
data_train = pd.read_csv('train_movie.csv')#Importing dataset for training
data_test = pd.read_csv('test_movie.csv')#Importing dataset for testing
col=list(data_train.columns)
user_index=col[0]
movie_index=col[1]
users=np.unique(data_train[col[0]])
movies=np.unique(data_train[col[1]])
print("Number of unique users: %d movies: %d"%(len(users),len(movies)))
dic_user={}
for user in users:
    temp={}
    movies_1=list(data_train.values[data_train[user_index]==user,1])
    ratings_1=list(data_train.values[data_train[user_index]==user,2])
    for i,m in enumerate(movies_1):
        temp[m]=ratings_1[i]
    dic_user[user]=temp

Number of unique users: 541 movies: 1211


## 1.2 Implement User-based Collaborative Filtering

In this part, you will implement the basic User–User Collaborative Filtering algorithm introduced in the class. Given the ratings by each user, you are going to try different methods in calculating the similarities between users. You will use equation `(c)` in `week06rec.pdf` (Page 40) to aggregate ratings. Set k = 0.05. Just consider all users as neighbors. That is, while predicting how user $u$ will rate item $i$, $\widehat{C}$ includes all users who have ratings on i in the training set.

*For this memory-based algorithm, you can only rely on the ratings in training set to predict for the testing set. That is, you assume that you don't know any ratings information in the test set except that when you evalaute your model.*

### Cosine Similarity

First, you will try to calculate the similarity between users with cosine similary following the equation on page 16 of [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). And then you need to predict the rating for each (user, movie) tuple in the test set. *Note: you don't need to subtract user mean baseline from the ratings prior to computing the similarity.*

In [2]:
# Your Code Here...
# Predict for test set
dic_sim={}
user_comm={}
for i,user1 in enumerate(users):
    s=set(list(dic_user[user1].keys()))
    ratings_all_1=list(data_train.values[data_train[user_index]==user1,2])
    for j in range(i+1,len(users)):
        user2=users[j]
        common=list(s.intersection(list(dic_user[user2].keys())))
        user_comm[(user1,user2)]=common
        ratings_all_2=list(data_train.values[data_train[user_index]==user2,2])
        if(len(common)==0):
            num=0
        else:
            ratings_1=[dic_user[user1][m] for m in common]
            ratings_2=[dic_user[user2][m] for m in common]
            num=np.dot(ratings_1,ratings_2)
        den=math.sqrt(np.dot(ratings_all_1, ratings_all_1))*math.sqrt(np.dot(ratings_all_2, ratings_all_2))
        dic_sim[(user1,user2)] = num/den
dic_mean_rat={}
def findmean():
    dic_mean={}
    users_test=np.unique(data_test[col[0]])
    for user in users_test:
            rat=data_train.values[data_train[user_index]==user,2]
            dic_mean[user]=(np.sum(rat)/len(rat))
    return dic_mean
dic_mean_rat=findmean()

In [6]:
def pred(fun):
    pred=[]
    k=0.05
    for row in data_test.values:
        user=row[0]
        movie=row[1]
        x=0
        for user2 in users:
            if(user!=user2 and (movie in dic_user[user2].keys())):
                if ((user2>user) and (fun[(user,user2)]!=0)):
                    x+=(fun[(user,user2)]*(dic_user[user2][movie]-dic_mean_rat[user2]))
                elif((user2<user)and (fun[(user2,user)]!=0)):
                    x+=(fun[(user2,user)]*(dic_user[user2][movie]-dic_mean_rat[user2]))
        temp_pred=dic_mean_rat[user]+k*x
        pred.append(temp_pred)
    return pred
pred_cos=pred(dic_sim)

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [7]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test set
from sklearn import metrics
print("Cosine Similarity: MAE-%0.3f RMSE-%0.3f"%(metrics.mean_absolute_error(data_test.values[:,2],pred_cos),math.sqrt(metrics.mean_squared_error(data_test.values[:,2],pred_cos))))

Cosine Similarity: MAE-0.690 RMSE-0.899


### Pearson correlation

Then, you will try to calculate the similarity between users with pearson correlation following `week06rec.pdf` (Page 37). And then you need to predict the rating for each (user, movie) tuple in the test set.

In [8]:
# Your Code Here...
# Predict for test set
dic_sim_p={}
count=0
for i,user1 in enumerate(users):
    for j in range(i+1,len(users)):
        user2=users[j]
        common=user_comm[(user1,user2)]
        if len(common)==0:
            dic_sim_p[(user1,user2)]=0
        else:
            ratings_1=[dic_user[user1][m] for m in common]
            ratings_2=[dic_user[user2][m] for m in common]
            ratings_1=[(r-dic_mean_rat[user1]) for r in ratings_1]
            ratings_2=[(r-dic_mean_rat[user2]) for r in ratings_2]
            if(np.dot(ratings_1, ratings_1)==0 or np.dot(ratings_2, ratings_2)==0):
                dic_sim_p[(user1,user2)] =0
            else:
                den=math.sqrt(np.dot(ratings_1, ratings_1))*math.sqrt(np.dot(ratings_2, ratings_2))
                num=np.dot(ratings_1,ratings_2)
                dic_sim_p[(user1,user2)] = num/den

In [9]:
predictions=pred(dic_sim_p)
print(predictions)

[4.345342125701321, 4.33293350920042, 4.377390696314047, 4.452611744485244, 4.442938316237413, 4.553229125584008, 4.433225913100281, 4.957562464884944, 4.486931400633419, 4.330476516250518, 4.516054860836224, 4.205221477916872, 4.497891427040479, 4.680004891330315, 4.690933983886543, 3.7166774550835298, 4.520586934640404, 4.915830191300064, 4.48440040371903, 4.5355020288882475, 4.39414169765368, 4.332775273437752, 4.456629083093352, 4.364051559086142, 4.7639440651232645, 5.563096399234896, 4.867965893336193, 4.61189511911521, 4.638414398219314, 4.587993626375483, 5.0754150767123285, 4.486627890723994, 4.330377663748435, 4.723466718902735, 4.359519509407989, 4.360302043922444, 4.409218100631672, 4.395032813508525, 4.505836354996945, 4.401506114591026, 4.988499930173289, 4.51239494113976, 4.508773955131118, 5.323342888519186, 4.503210243603306, 4.6335795771620605, 4.178078101167011, 4.27486340042827, 4.541474322305339, 4.454501173779751, 4.124017002198943, 5.118954043738536, 4.3744865684

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [10]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test
print("Pearson Coefficient: MAE-%0.3f MSE-%0.3f"%(metrics.mean_absolute_error(data_test.values[:,2],predictions),math.sqrt(metrics.mean_squared_error(data_test.values[:,2],predictions))))


Pearson Coefficient: MAE-0.692 MSE-0.898


### Pearson correlation (varying the threshold)

In [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf), they observe that: 

> Pearson correlation suffers from computing high similarity
between users with few ratings in common. This can be alleviated by setting a threshold on the number of co-rated items
necessary for full agreement (correlation of 1) and scaling the
similarity when the number of co-rated items falls below this
threshold.

So now revise your Pearson to consider a threshold. Try several values and report for one that you think is appropriate.

In [11]:
# Your Code Here...
# Predict for test set
def thresh_p(threshold):
    dic_sim_p_t={}
    for i,user1 in enumerate(users):
        for j in range(i+1,len(users)):
            user2=users[j]
            common=user_comm[(user1,user2)]
            if len(common)==0:
                dic_sim_p_t[(user1,user2)]=0
            else:
                ratings_1=[dic_user[user1][m] for m in common]
                ratings_2=[dic_user[user2][m] for m in common]
                ratings_1=[(r-dic_mean_rat[user1]) for r in ratings_1]
                ratings_2=[(r-dic_mean_rat[user2]) for r in ratings_2]
                if(np.dot(ratings_1, ratings_1)==0 or np.dot(ratings_2, ratings_2)==0):
                    dic_sim_p_t[(user1,user2)] =0
                else:
                    den=math.sqrt(np.dot(ratings_1, ratings_1))*math.sqrt(np.dot(ratings_2, ratings_2))
                    num=np.dot(ratings_1,ratings_2)
                    dic_sim_p_t[(user1,user2)] = min(len(common)/threshold,1)*num/den
    return dic_sim_p_t

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [12]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

d_sim_p_thresh={}
thr=list(range(5,13))+[15,20,30]
for t in thr:
    d_sim_p_thresh=thresh_p(t)
    predictions_thresh=pred(d_sim_p_thresh)
    print("Pearson Coefficient for threshold=%d: MAE-%0.3f MSE-%0.3f"%(t,metrics.mean_absolute_error(data_test.values[:,2],predictions_thresh),math.sqrt(metrics.mean_squared_error(data_test.values[:,2],predictions_thresh))))


Pearson Coefficient for threshold=5: MAE-0.687 MSE-0.893
Pearson Coefficient for threshold=6: MAE-0.686 MSE-0.892
Pearson Coefficient for threshold=7: MAE-0.686 MSE-0.891
Pearson Coefficient for threshold=8: MAE-0.686 MSE-0.891
Pearson Coefficient for threshold=9: MAE-0.685 MSE-0.890
Pearson Coefficient for threshold=10: MAE-0.685 MSE-0.890
Pearson Coefficient for threshold=11: MAE-0.685 MSE-0.890
Pearson Coefficient for threshold=12: MAE-0.685 MSE-0.890
Pearson Coefficient for threshold=15: MAE-0.685 MSE-0.891
Pearson Coefficient for threshold=20: MAE-0.687 MSE-0.892
Pearson Coefficient for threshold=30: MAE-0.690 MSE-0.896


What do you observe? How different are your results for the original Pearson Correlation approach vs. the thresholded version vs. the Cosine Similarity approach? Why do you think this is? *provide a brief (1-2 paragraph) discussion based on these questions.*

### Observation on Cosine Similarity vs. Original Pearson Correlation approach:-
From the above code block, we observe that the average number of co-rated items is around 9.So, on average, we need a threshold of 9 for full agreement (correlation of 1), and we should scale down the similarity when the number of co-rated items falls below this threshold. We observe that on setting a threshold around 9, we achieve lower RMSE(0.890 as compared to 0.898 without threshold). The same deteriorates as we increase the threshold beyond 13.

In [208]:
print("Average length of number of co-rated items for all the user pairs=%0.2f"%(np.sum([len(value) for value in user_comm.values()])/len([len(value) for value in user_comm.values()])))
    

Average length of number of co-rated items for all the user pairs=8.83


### Observation on Thresholded Pearson Correlation approach vs. Original Pearson Correlation approach:-
From the above code block, we observe that the average number of co-rated items is around 9.So, on average, we need a threshold of 9 for full agreement (correlation of 1), and we should scale down the similarity when the number of co-rated items falls below this threshold. We observe that on setting a threshold around 9, we achieve lower RMSE(0.890 as compared to 0.898 without threshold). The same deteriorates as we increase the threshold beyond 13.

# Part 2. MF

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set. *You are expected to get a lower MAE and RMSE compared with the results in Part 1.*

In [13]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
def getInitialMatrix():
    A = data_train.pivot_table(columns=[col[1]], index=[col[0]], values=[col[2]]).fillna(-1).values

    print(" 'A' matrix shape is", A.shape)
    print ("Getting 'R' Binary Matrix of rating or no rating...")
    R = A>=0; R[R == True] = 1; R[R == False] = 0; R = R.astype(np.float64, copy=False)
    return A, R
def runALS(A, R, n_factors, n_iterations, lambda_):
    '''
    Runs Alternating Least Squares algorithm in order to calculate matrix.
    :param A: User-Item Matrix with ratings
    :param R: User-Item Matrix with 1 if there is a rating or 0 if not
    :param n_factors: How many factors each of user and item matrix will consider
    :param n_iterations: How many times to run algorithm
    :param lambda_: Regularization parameter
    :return:
    '''
    print ("Initiating ")
    n, m = A.shape; 
    Users = 5 * np.random.rand(n, n_factors)
    Items = 5 * np.random.rand(n_factors, m)

    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R)

    MSE_List = []
    for iter in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(np.dot(Items, np.dot(np.diag(Ri), Items.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Items, np.dot(np.diag(Ri), A[i].T))).T
        print ("Error after solving for User Matrix:", get_error(A, Users, Items, R))

        for j, Rj in enumerate(R.T):
            Items[:,j] = np.linalg.solve(np.dot(Users.T, np.dot(np.diag(Rj), Users)) + lambda_ * np.eye(n_factors),
                                     np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])))
        print ("Error after solving for Item Matrix", get_error(A, Users, Items, R))
        print(np.dot(Users[0],Items[:,0]))
#                 X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
#                                     np.dot(Y, Q.T)).T
#                 Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
#                                     np.dot(X.T, Q))

        MSE_List.append(get_error(A, Users, Items, R))
        print ('%sth iteration is complete...' % iter)

    print (MSE_List)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(range(1, len(MSE_List) + 1), MSE_List); plt.ylabel('Error'); plt.xlabel('Iteration')
    plt.title('Python Implementation MSE by Iteration \n with %d users and %d movies' % A.shape);
    plt.savefig('Python MSE Graph.pdf', format='pdf')
    plt.show()
    return Users,Items

A, R = getInitialMatrix()
U,I=runALS(A, R, n_factors = 3, n_iterations = 40, lambda_ = 3)

 'A' matrix shape is (541, 1211)
Getting 'R' Binary Matrix of rating or no rating...
Initiating 
Error after solving for User Matrix: 2.109587249057566
Error after solving for Item Matrix 0.9933076726330692
4.497231909901218
0th iteration is complete...
Error after solving for User Matrix: 0.7465365917316296
Error after solving for Item Matrix 0.8105875868118068
4.661488541223084
1th iteration is complete...
Error after solving for User Matrix: 0.681646508014434
Error after solving for Item Matrix 0.7393794149551385
4.6070390847671066
2th iteration is complete...
Error after solving for User Matrix: 0.6512259115947923
Error after solving for Item Matrix 0.6979141336529959
4.597193451308455
3th iteration is complete...
Error after solving for User Matrix: 0.6349186038682836
Error after solving for Item Matrix 0.6732494151146283
4.61194087858889
4th iteration is complete...
Error after solving for User Matrix: 0.6253411793864737
Error after solving for Item Matrix 0.6568568493088994
4.63

<Figure size 640x480 with 1 Axes>

In [14]:
final_mf=np.dot(U,I)
def pred_mf():
    pred=[]
    for row in data_test.values:
        user=row[0]
        movie=row[1]
        pred.append(final_mf[user][movie])
    return pred
predictions_mf=pred_mf()
# Report Mean Absolute Error and Root Mean Squared Error for test
print("MF with ALS: MAE-%0.3f MSE-%0.3f"%(metrics.mean_absolute_error(data_test.values[:,2],predictions_mf),math.sqrt(metrics.mean_squared_error(data_test.values[:,2],predictions_mf))))

MF with ALS: MAE-0.679 MSE-0.879


Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

I chose **Alternating Least Squeares** method.


**Advantages**: The ALS successfully breaks down the user-movie matrix into latent factor models and performs better than Pearson Correlation and Cosine Similarity while handling missing ratings.We are minimizing the entire loss function at once, but, only twiddling half the parameters. That's because the optimization has an easy algebraic solution,if half the parameters are fixed. So we fix half, recompute the other half, and repeat. There is no gradient in the optimization step since each optimization problem is convex and doesn't need an approximate approach.
**Advantages**: It cannot handle a cold-start problem and cannot make any predictions for a new user without retraining the model.Moreover, the naive minimization of a ranking objective function is typically expensive.

# Part 3. Extension

Given your results in the previous parts, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system, and so on. *As in the last homework, you can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than both of your CF and MF (of course we will also judge whether what you do here is reasonable or not). Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [17]:
d_sim_p_thresh=thresh_p(9)
predictions_thresh_1=pred(d_sim_p_thresh)

In [22]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test
avg_mov_rat={}
avg_u_cor={}

def findavgrat(m):
    if m not in avg_mov_rat.keys():
        ratings=list(data_train.values[data_train[col[1]]==m,2])
        avg_mov_rat[m]=np.mean(ratings)
        return avg_mov_rat[m]
    else:
        return avg_mov_rat[m]
def pred_best():
    pred=[]
    for i,row in enumerate(data_test.values):
        user=row[0]
        movie=row[1]
        temp=((0.6*final_mf[user][movie])+(0.4*predictions_thresh_1[i]))
        pred.append(temp)
    return pred

predictions_best=pred_best()
print("MF with New Method: MAE-%0.3f MSE-%0.4f"%(metrics.mean_absolute_error(data_test.values[:,2],predictions_best),math.sqrt(metrics.mean_squared_error(data_test.values[:,2],predictions_best))))


MF with New Method: MAE-0.660 MSE-0.8567


Please explain what you do to improve the recommendation in 1-2 paragraphs.

I took a weighted average of the predictions from the MF and the Pearson Correlation. A bigger emphasis is given on the MF based prediction as it takes care of the individual user-item relation. The addition of the Pearson Correlation coefficient also takes into account the collaborative factor, i.e. the prediction based on the similar users. I think the collaboration predicition should have a lesser weightage than the former as there may be users who do not concur with the crowd. 

This reduces the RMSE from 0.898 to 0.856. These weights can be learned through a deep learning network. Here I have just hand tuned the weights for evaluating the method.

https://github.com/mickeykedia/Matrix-Factorization-ALS

https://www.researchgate.net/publication/254464370_Alternating_least_squares_for_personalized_ranking